In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train.drop(columns = 'id',inplace=True)
test = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")
test_id = test['id']
test.drop(columns = 'id',inplace=True)
train.info()

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 22.9+ MB


In [2]:
for col in train.columns:
    print(train[col].value_counts(),"\n","=="*30)

Brand
Adidas          60077
Under Armour    59992
Nike            57336
Puma            56814
Jansport        56076
Name: count, dtype: int64 
Material
Polyester    79630
Leather      73416
Nylon        70603
Canvas       68004
Name: count, dtype: int64 
Size
Medium    101906
Large      98643
Small      92856
Name: count, dtype: int64 
Compartments
1.0     31568
4.0     31498
2.0     30595
3.0     30293
7.0     30158
9.0     29965
5.0     29959
10.0    29815
8.0     28909
6.0     27240
Name: count, dtype: int64 
Laptop Compartment
Yes    148342
No     144214
Name: count, dtype: int64 
Waterproof
Yes    148077
No     144873
Name: count, dtype: int64 
Style
Messenger    100031
Tote          97438
Backpack      94561
Name: count, dtype: int64 
Color
Pink     51690
Gray     50280
Blue     48097
Red      47219
Green    46386
Black    46378
Name: count, dtype: int64 
Weight Capacity (kg)
5.000000     4286
30.000000     185
11.898250     123
22.898382     116
14.908437     101
             ..

In [3]:
%%time
def rmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean((y_true - y_pred) ** 2))


def eda(df):
    if 'Size' in df.columns:
        df['Size'].replace({"Small": 0, "Medium": 1, "Large": 2}, inplace=True)

    categorical_columns = ['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
    
    existing_categorical_columns = [col for col in categorical_columns if col in df.columns]

    df = pd.get_dummies(df, columns=existing_categorical_columns, dummy_na=True)

    df['Weight Capacity (kg)'].fillna(df['Weight Capacity (kg)'].mode()[0], inplace=True)

    from sklearn.ensemble import RandomForestRegressor

    train = df.dropna(subset=['Size'])  # Rows where 'Size' is not missing
    test = df[df['Size'].isna()]  # Rows where 'Size' is missing
    
    X_train = train.drop(columns=['Size'])
    y_train = train['Size']
    X_test = test.drop(columns=['Size'])
    
    # Train Random Forest Regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=16)
    rf.fit(X_train, y_train)
    
    # Predict missing values
    df.loc[df['Size'].isna(), 'Size'] = rf.predict(X_test)

    return df


train_df = eda(train)
test_df = eda(test)

<timed exec>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


<timed exec>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
<timed exec>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in panda

CPU times: user 10min 3s, sys: 2.72 s, total: 10min 6s
Wall time: 10min 7s


In [4]:
%%time
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Size                    300000 non-null  float64
 1   Compartments            300000 non-null  float64
 2   Weight Capacity (kg)    300000 non-null  float64
 3   Price                   300000 non-null  float64
 4   Brand_Adidas            300000 non-null  bool   
 5   Brand_Jansport          300000 non-null  bool   
 6   Brand_Nike              300000 non-null  bool   
 7   Brand_Puma              300000 non-null  bool   
 8   Brand_Under Armour      300000 non-null  bool   
 9   Brand_nan               300000 non-null  bool   
 10  Material_Canvas         300000 non-null  bool   
 11  Material_Leather        300000 non-null  bool   
 12  Material_Nylon          300000 non-null  bool   
 13  Material_Polyester      300000 non-null  bool   
 14  Material_nan        

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet,Ridge,SGDRegressor,BayesianRidge,TheilSenRegressor,RANSACRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [6]:
x = train_df.drop(columns = ['Price'])
y = train_df['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=16)

In [7]:
%%time
def model_list(models, random_state_num, x_train, x_test, y_train, y_test):
    rmse_score_dict = {}
    for model_class in models:
        model = model_class(random_state=random_state_num)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        
        model_name = model_class.__name__
        
        rmse_score_dict[model_name] = rmse(y_test, y_pred)
    
    return rmse_score_dict

models = [DecisionTreeRegressor, GradientBoostingRegressor, RandomForestRegressor, XGBRegressor, LGBMRegressor]

rmse_scores = model_list(models, 16, x_train, x_test, y_train, y_test)

print("RMSE Scores for different models:")
for model_name, score in rmse_scores.items():
    print(f"{model_name}: {score:.4f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 410
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 31
[LightGBM] [Info] Start training from score 81.429372
RMSE Scores for different models:
DecisionTreeRegressor: 56.1731
GradientBoostingRegressor: 38.9460
RandomForestRegressor: 40.1089
XGBRegressor: 39.1460
LGBMRegressor: 38.9628
CPU times: user 4min 47s, sys: 2.23 s, total: 4min 50s
Wall time: 4min 42s


In [8]:
%%time
def linear_model_list(models, x_train, x_test, y_train, y_test):
    rmse_score_dict = {}
    for model_class in models:
        model = model_class()
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        
        model_name = model_class.__name__
        
        rmse_score_dict[model_name] = rmse(y_test, y_pred)
    
    return rmse_score_dict

models = [LinearRegression, Lasso, ElasticNet,Ridge,SGDRegressor,BayesianRidge,TheilSenRegressor,RANSACRegressor]

rmse_scores = linear_model_list(models, x_train, x_test, y_train, y_test)

print("RMSE Scores for different models:")
for model_name, score in rmse_scores.items():
    print(f"{model_name}: {score:.4f}")

RMSE Scores for different models:
LinearRegression: 38.9626
Lasso: 38.9848
ElasticNet: 38.9839
Ridge: 38.9626
SGDRegressor: 39.3042
BayesianRidge: 38.9632
TheilSenRegressor: 39.0020
RANSACRegressor: 47.5622
CPU times: user 1min 18s, sys: 17.7 s, total: 1min 36s
Wall time: 1min 33s


In [9]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 13,
    'reg_alpha': 0.01,
    'lambda_l2': 0.01,  
    'min_child_samples' : 32,
    'colsample_bytree': 0.93,
    'subsample': 0.7, 
    'seed': 42,
    'verbose': -1,
    'device' : 'cpu' 
}
LGBM_model = LGBMRegressor(**params)
LGBM_model.fit(x_train, y_train)
y_pred = LGBM_model.predict(x_test)

# Calculate RMSLE score
score = rmse(y_test, y_pred)
print(f"RMSE for LGBMRegressor: {score:.4f}")

RMSE for LGBMRegressor: 39.1061


In [10]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 13,
    'reg_alpha': 0.01,
    'lambda_l2': 0.01,  
    'min_child_samples' : 32,
    'colsample_bytree': 0.93,
    'subsample': 0.7, 
    'seed': 42,
    'verbose': -1,
    'device' : 'cpu' 
}
LGBM_model = LGBMRegressor(**params)
LGBM_model.fit(x_train, y_train)
LGBM_preds = LGBM_model.predict(test_df)
predictions = pd.DataFrame({
    'id': test_id,
    'Price': LGBM_preds
})
predictions.to_csv('LGBM_ans.csv', index=False)

In [11]:
%%time
TheilSenRegressor_model = TheilSenRegressor()
TheilSenRegressor_model.fit(x_train, y_train)
TSR_preds = TheilSenRegressor_model.predict(test_df)
predictions = pd.DataFrame({
    'id': test_id,
    'Price': TSR_preds
})
predictions.to_csv('TSR_ans.csv', index=False)

CPU times: user 1min 6s, sys: 16.2 s, total: 1min 22s
Wall time: 1min 22s
